In [1]:
import torch
import torch.nn as nn
from model import GlobalPredictor

In [2]:
import os

In [3]:
import pickle as pk

In [4]:
global_predictor = torch.load('./results_tokyo/global_predictor_fl.pytorch').cuda(0)

In [5]:
date_list = [(2012, m, d) for m in range(12, 13) for d in range(1, 32)] + [(2013, m, d) for m in range(1, 3) for d in range(1, 32)]

In [6]:
T = 96
dT = 4
batch_size = 256

In [7]:
avg_loss_dict = dict({})

for y, m, d in date_list:
    filename = '/data/fan/UsersInTokyoProcessed/{:04d}{:02d}{:02d}_interp.pk'.format(y, m, d)
    if not os.path.isfile(filename):
        continue
    print(filename)
    
    avg_loss = 0.0
    with open(filename, 'rb') as f:
        data = pk.load(f)
        data = torch.LongTensor(list(data.values())).cuda(0)
        with torch.no_grad():
            for t in range(T - 2 * dT):
                avg_loss_t = 0.0
                for i in range(0, data.shape[0], batch_size):
                    data_x = data[i: i + batch_size, t: t + dT]
                    data_t = torch.zeros_like(data_x) + t
                    data_y = data[i: i + batch_size, t + 2 * dT - 1]
                    loss = global_predictor(data_x, data_t, data_y).item()
                    avg_loss += loss
                    avg_loss_t += loss
        
                avg_loss_t /= data.shape[0]
                avg_loss_dict[(y, m, d, t)] = avg_loss_t
                
            avg_loss /= (data.shape[0] * (T - 2 * dT))
            avg_loss_dict[(y, m, d)] = avg_loss
            print('Evaluation: {:.4f}'.format(avg_loss))

/data/fan/UsersInTokyoProcessed/20121201_interp.pk
Evaluation: 1.7679
/data/fan/UsersInTokyoProcessed/20121202_interp.pk
Evaluation: 1.6195
/data/fan/UsersInTokyoProcessed/20121203_interp.pk
Evaluation: 1.7758
/data/fan/UsersInTokyoProcessed/20121204_interp.pk
Evaluation: 1.7931
/data/fan/UsersInTokyoProcessed/20121205_interp.pk
Evaluation: 1.8095
/data/fan/UsersInTokyoProcessed/20121206_interp.pk
Evaluation: 1.8188
/data/fan/UsersInTokyoProcessed/20121207_interp.pk
Evaluation: 1.8377
/data/fan/UsersInTokyoProcessed/20121208_interp.pk
Evaluation: 1.7587
/data/fan/UsersInTokyoProcessed/20121209_interp.pk
Evaluation: 1.6232
/data/fan/UsersInTokyoProcessed/20121210_interp.pk
Evaluation: 1.7955
/data/fan/UsersInTokyoProcessed/20121211_interp.pk
Evaluation: 1.8142
/data/fan/UsersInTokyoProcessed/20121212_interp.pk
Evaluation: 1.8162
/data/fan/UsersInTokyoProcessed/20121213_interp.pk
Evaluation: 1.8303
/data/fan/UsersInTokyoProcessed/20121214_interp.pk
Evaluation: 1.8445
/data/fan/UsersInTok

In [8]:
with open('./results_tokyo/evaluation_global_fl_nll.pk', 'wb') as f:
    pk.dump(avg_loss_dict, f)